# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import time
import re

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [4]:
driver.find_element_by_id('checkbox').click() #for the checkbox
driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]').click() #to click enter

Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [5]:
driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [6]:
from selenium.webdriver.support.ui import Select #easier with select
select = Select(driver.find_element_by_name('slcJurisdiction'))
select.select_by_visible_text('Statewide')
driver.find_element_by_id('txtTradeName').send_keys('VAP%')

And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

In [7]:
#Nothing there to scrape the select button, so I use the xpath
button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]')

In [8]:
driver.execute_script("arguments[0].scrollIntoView(true)", button) #scroll to the search results
button.click()

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [9]:
#Store's name
first_name = driver.find_element_by_class_name('searchlistitem').text
first_name

'VAPE IT STORE II'

In [10]:
names = driver.find_elements_by_class_name('searchlistitem')
len(names)

5

In [11]:
for name in names:
    print(name.text)

VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


In [12]:
#Better: search with 'searchfieldtitle
results = driver.find_elements_by_class_name('searchfieldtitle')
for result in results:
    print(result.text)

1. VAPE IT STORE II
2. VAPE IT STORE I
3. VAPEPAD THE
4. VAPE FROG Pending *
5. VAPE FROG Pending *


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [13]:
#to open the detail page:
select_button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[3]/a/img')
select_button.click()
#to go back to the search page: driver.back()

In [14]:
#To get the url of the first detail page:
url = driver.find_element_by_tag_name('a').get_attribute('href')
print(url)

https://jportal.mdcourts.gov/license/pbIndex.jsp


Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

In [15]:
#Mailing address
#Location address
#License information (you can keep it as one field)
#Total amount paid
#Issued by


results = driver.find_elements_by_class_name('searchfieldtitle')
time.sleep(1)

for result in results:
    print(driver.find_element_by_class_name('td').text)
        

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"class name","selector":"td"}
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.10.5 x86_64)


### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [ ]:
#tried it above. I think the tips are are more confusing than helpful. I don't get the logic of being told what to try out with at the same time telling me that it won't work. So I move directly to the while loop.

### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [21]:
df_vape=[] #my empty list for all the names and url's
try:
    while True: #while so it doesn't stop
    
        results=driver.find_elements_by_class_name('searchfieldtitle')
        for result in results:
            vap={} #my empty dict
            company=result.find_element_by_class_name('searchlistitem') # my way of scraping the names from the detail page
            vap['name']=company.text #storing the names in the dict
            print(company.text)
            try:
                
                search_button = result.find_element_by_tag_name('a')
                vap['url']=search_button.get_attribute('href') #storing the url's in the dict
                print('url is done')
                print('-----------------')
            except:
                print('no url')
            df_vape.append(vap) #fill my empty list
            
        button = driver.find_element_by_partial_link_text('Next') #moving on and doing the same with the other pages
        driver.execute_script("arguments[0].scrollIntoView(true)", button)
        button.click() 
        print('scraping next page') 
        
            
except:
    print('done')


VAPE IT STORE II
url is done
-----------------
VAPE IT STORE I
url is done
-----------------
VAPEPAD THE
url is done
-----------------
VAPE FROG
no url
VAPE FROG
no url
scraping next page
VAPE LOFT THE
url is done
-----------------
VAPE N CIGAR
url is done
-----------------
VAPE DOJO
url is done
-----------------
VAPE HAVEN
no url
VAPORS LOUNGE
no url
scraping next page
VAPORFI SOUTHSIDE
url is done
-----------------
VAPE DO JO
url is done
-----------------
VAPERTETMD
no url
VAPERTEKMD
url is done
-----------------
VAPORIOT
no url
scraping next page
VAPE PLUS
url is done
-----------------
VAPE SOCIAL
url is done
-----------------
VAPE JUNGLE
url is done
-----------------
VAPE TIME
no url
VAPEBAR EXPESS
no url
scraping next page
VAPEWERKS
url is done
-----------------
VAPIST MD
url is done
-----------------
VAPOR 99
no url
VAPOR ALLEY
url is done
-----------------
VAPOR EMPORIUM
url is done
-----------------
scraping next page
VAPOR VILLA
url is done
-----------------
VAPOR VILLA
url is

### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [22]:
df_vape

[{'name': 'VAPE IT STORE II',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=yfC5KNy1F%2Fk%3D'},
 {'name': 'VAPE IT STORE I',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=OJLH%2BT6iV%2Fc%3D'},
 {'name': 'VAPEPAD THE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pEQ4SUAlRSY%3D'},
 {'name': 'VAPE FROG'},
 {'name': 'VAPE FROG'},
 {'name': 'VAPE LOFT THE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=bX%2BtwhKmo0E%3D'},
 {'name': 'VAPE N CIGAR',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=AiJyZiSYX34%3D'},
 {'name': 'VAPE DOJO',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=T%2FtJ6w3dfeA%3D'},
 {'name': 'VAPE HAVEN'},
 {'name': 'VAPORS LOUNGE'},
 {'name': 'VAPORFI SOUTHSIDE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=AcHgFUJy%2BGw%3D'},
 {'name': 'VAPE DO JO',
  'url': 'https://jportal.mdcourts.gov/license/pb

### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [24]:
df = pd.DataFrame(df_vape)
df

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE FROG,NaN
4,VAPE FROG,NaN
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...
8,VAPE HAVEN,NaN
9,VAPORS LOUNGE,NaN


In [27]:
df.to_csv("vape-shops-basic.csv", index=False)

# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [28]:
pd.set_option('display.max_colwidth', -1)

In [29]:
df = pd.read_csv('vape-shops-basic.csv', na_values=['NaN'])
df.head(10)

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=yfC5KNy1F%2Fk%3D
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=OJLH%2BT6iV%2Fc%3D
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pEQ4SUAlRSY%3D
3,VAPE FROG,NaN
4,VAPE FROG,NaN
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=bX%2BtwhKmo0E%3D
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=AiJyZiSYX34%3D
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=T%2FtJ6w3dfeA%3D
8,VAPE HAVEN,NaN
9,VAPORS LOUNGE,NaN


### Use Selenium to visit that page

In [ ]:
#Sorry, but I just can't spend more time on this vape notebook...it feels like I'm stuck in a loop myself.

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.